In [1]:
import gspread
from google.oauth2.service_account import Credentials
from hashlib import sha256
from hmac import HMAC
from sys import api_version
import requests
from datetime import datetime
import urllib.parse
import pandas as pd
import os
import pytz

In [2]:
os.environ["GOOGLE_VALOREO_SOURCING_CREDENTIALS"]="../consol_inventario/valoreo_sourcing.json"
path_to_credentials_file = os.environ.get('GOOGLE_VALOREO_SOURCING_CREDENTIALS')
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(path_to_credentials_file, scopes=scope)

In [3]:
parameters = {
    'Action':           'GetProducts',
    'Filter':           'all',
    'Format':           'JSON',
    'GlobalIdentifier': 0,
    #'SkuSellerList':    [7502308501508],
    'Timestamp':        datetime.now().isoformat(timespec='seconds'),
    'UserID':           'roberto@valoreo.com',
    'Version':          '1.0'
}

# NEEDED TO CREATE SIGNATURE
parameters_tuples   = sorted(parameters.items())
concatenated_params = urllib.parse.urlencode(parameters_tuples)

#api_key = '8c74684ef88b39b0a9970492733ba0812e47ca06' # Ismael
api_key = '7c00e83f035aa41d176d999fa21afd6035498147' #Roberto

# CREATE SIGNATURE
signature = HMAC(api_key.encode('ascii'),
                 concatenated_params.encode('ascii'), sha256).hexdigest()
parameters['Signature'] = signature


# PARAMETERS + SIGNATURE
str_parameters = urllib.parse.urlencode( parameters )

# REQUEST

base_url = "https://sellercenter-api.linio.com.co"
headers  = {"accept": "application/*"}

r = requests.get( f'{base_url}/?{str_parameters}', headers=headers)

api_response_json = r.json()

In [4]:
df = pd.json_normalize( api_response_json['SuccessResponse']['Body']['Products']['Product'] )
try:
    df = df[ ['SellerSku','ShopSku', 'Quantity','FulfillmentByNonSellable','FulfillmentBySellable'] ]
except:
    df = df[ ['SellerSku','ShopSku', 'Quantity','FulfillmentBySellable'] ]
try:
    df['FulfillmentByNonSellable'] = df['FulfillmentByNonSellable'].fillna(0).astype(int)
except:
    pass
df['FulfillmentBySellable'] = df['FulfillmentBySellable'].fillna(0).astype(int)
df['Quantity'] = df['Quantity'].fillna(0).astype(int)

In [14]:
df['EAN_TEXT'] = "EAN"+("000000000"+df['SellerSku']).str[-14:]

In [18]:
linio_inv = df[['EAN_TEXT', 'Quantity', 'FulfillmentBySellable']]

In [19]:
local_tz = pytz.timezone('America/Mexico_City')
current_day = datetime.today().astimezone(local_tz).strftime('%Y%m%d')
linio_inv.to_csv(f'linio_inv.csv', index=False)
linio_inv.to_csv(f'{current_day}_linio_inv.csv', index=False)